In [1]:
# 모듈 불러오기
from Classes import BirdSoundDataset_te as DSET
from Classes import BirdRecognition as MODEL
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd
import torch
import json
import pickle

# Device 설정
if torch.cuda.is_available(): DEVICE = 'cuda'
else                        : DEVICE = 'cpu'

# 데이터셋, 모델 불러오기

In [2]:
with open('./scored_birds.json', 'r') as file:  # Bird classes for score
    classes = json.load(file)

testset = DSET(Path('./test_soundscapes'))      # test datset

model = MODEL(32000)

param_path = list(Path('./params').glob('*'))   # prameter list
param_path.sort()
param_path = param_path[-1]                     # last (newest) parameter
with open(param_path, 'rb') as file:
    param = pickle.load(file)                   # load parameter
model.load_state_dict(param.to(DEVICE))

<All keys matched successfully>

# 테스트

In [3]:
answers = { 'row_id': [], 'target': [] }

for data in tqdm(testset):
    inputs = data[1].tolist()
    
    for idx, input_ in enumerate(inputs):
        input_ = torch.Tensor(input_)
        out = model(input_).tolist()

        for bidx, class_ in enumerate(classes):
            answer_ = True if out[bidx] >= 0.5 else False
            id_ = f'{data[0]}_{class_}_{5 * (idx+1)}'
            answers['row_id'].append(id_)
            answers['target'].append(answer_)

answers = pd.DataFrame(answers)
answers.sort_values(by='row_id', inplace=True)
answers.to_csv('submission.csv', index=False)

  0%|          | 0/1 [00:00<?, ?it/s]